In [ ]:
import pandas as pd

In [ ]:
tenis_df = pd.read_csv("dane/Dane_ATP_2000-2024 d.csv", sep=';')

In [ ]:
tenis_df.head(5)

In [ ]:
# Usuwamy kolumny ktore naszym zdaniem nie sa potrzebne
columns_to_drop = [
    'tourney_id', 'tourney_name', 'draw_size', 'match_num', 'winner_seed',
    'winner_name', 'loser_seed', 'winner_ioc', 'loser_name', 'loser_seed',
    'loser_ioc', 'score', 'tourney_date'
]

tenis_df = tenis_df.drop(columns=columns_to_drop)
tenis_df.head(5)

In [ ]:
# Wizualizujemy rozklad roznych powierzchni
tenis_df = tenis_df.dropna(subset=['surface'])
surface = tenis_df['surface'].value_counts()
surface.plot(kind='pie', autopct='%1.1f%%', startangle=90)

In [ ]:
# Zamieniamy statystyki meczowe na ich rolling avereges z ostatnich 10 meczow
def calculate_rolling_averages(df, columns_for_rolling_averages, player):
    df = df.copy()

    for col in columns_for_rolling_averages:
        df[f'{player}_rolling_{col[2:]}'] = (
            df.groupby(f'{player}_id')[col]
            .transform(lambda x: x.rolling(window=10, min_periods=1).mean())
        )

    return df

columns_for_rolling_averages_winner = [
    'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced']

columns_for_rolling_averages_loser = [
    'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced'
]

tenis_df = calculate_rolling_averages(tenis_df, columns_for_rolling_averages_winner, 'winner')
tenis_df.drop(columns=columns_for_rolling_averages_winner, inplace=True)

tenis_df = calculate_rolling_averages(tenis_df, columns_for_rolling_averages_loser, 'loser')
tenis_df.drop(columns=columns_for_rolling_averages_loser, inplace=True)

tenis_df.head(5)


In [ ]:
print(tenis_df.columns)

In [ ]:
# Zmiana nazw kolumn winner -> player_1 i loser -> player_0
tenis_df.columns = tenis_df.columns.str.replace('winner', 'player1')
tenis_df.columns = tenis_df.columns.str.replace('loser', 'player0')

# Sprawdzenie zmian
print(tenis_df.columns)

# Dodaj kolumnę label (player_1 zawsze wygrywa)
tenis_df['label'] = 1
tenis_df.head(5)


In [ ]:
print(tenis_df.dtypes)

In [ ]:
# One-hot encoding dla wybranych kolumn
columns_to_encode = ['surface', 'tourney_level', 'round', 'player0_entry', 'player0_hand', 'player1_entry', 'player1_hand']
tenis_df = pd.get_dummies(tenis_df, columns=columns_to_encode, drop_first=True)
tenis_df.head(5)


In [ ]:
# Wyprintuj wiersze oraz kolumny, w których są NaN
null_columns = tenis_df.columns[tenis_df.isnull().any()]
print(f"Kolumny z brakującymi wartościami: {list(null_columns)}")
print("Wiersze z brakującymi wartościami:")
print(tenis_df[tenis_df.isnull().any(axis=1)])

In [ ]:
# Uzupelniamy nulle odpowiednimi wartosciami
# Kolumny związane z wzrostem i wiekiem zawodników uzupelniamy srednia badz mediana
tenis_df['player1_ht'] = tenis_df['player1_ht'].fillna(tenis_df['player1_ht'].mean())
tenis_df['player0_ht'] = tenis_df['player0_ht'].fillna(tenis_df['player0_ht'].mean())
tenis_df['player1_age'] = tenis_df['player1_age'].fillna(tenis_df['player1_age'].median())
tenis_df['player0_age'] = tenis_df['player0_age'].fillna(tenis_df['player0_age'].median())

# Uzywamy mediany dla czasu trwania meczu
tenis_df['minutes'] = tenis_df['minutes'].fillna(tenis_df['minutes'].median())

# Dla rankingu i punktow rankingowych zakladamy ze zawodnik nie lapie sie do rangkingu ATP ktory w datasecie konczy sie na okolo 1350 i 1 punkcie atp, raning ustawiamy na 1500 a punkty na 0
tenis_df['player1_rank'] = tenis_df['player1_rank'].fillna(1500)  # Ranking gracza 1 na 1500
tenis_df['player0_rank'] = tenis_df['player0_rank'].fillna(1500)  # Ranking gracza 0 na 1500
tenis_df['player1_rank_points'] = tenis_df['player1_rank_points'].fillna(0)  # Punkty rankingowe gracza 1 na 0
tenis_df['player0_rank_points'] = tenis_df['player0_rank_points'].fillna(0)  # Punkty rankingowe gracza 0 na 0

# Dla kolumn rollingowych korzystamy z mediany
rolling_features = [
    'player1_rolling_ace', 'player1_rolling_df', 'player1_rolling_svpt',
    'player1_rolling_1stIn', 'player1_rolling_1stWon', 'player1_rolling_2ndWon',
    'player1_rolling_SvGms', 'player1_rolling_bpSaved', 'player1_rolling_bpFaced',
    'player0_rolling_ace', 'player0_rolling_df', 'player0_rolling_svpt',
    'player0_rolling_1stIn', 'player0_rolling_1stWon', 'player0_rolling_2ndWon',
    'player0_rolling_SvGms', 'player0_rolling_bpSaved', 'player0_rolling_bpFaced'
]

# Uzupełnienie brakujących danych wartościami mediany
for col in rolling_features:
    tenis_df[col] = tenis_df[col].fillna(tenis_df[col].median())

# Sprawdzenie, czy wszystkie brakujące wartości zostały wyeliminowane
print("Czy nadal istnieją brakujące wartości w danych?", tenis_df.isnull().sum().sum() > 0)

In [ ]:
tenis_df.to_csv('dane/gotowe_dane.csv', sep=';', index=False)